<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-you-run-this-notebook" data-toc-modified-id="Before-you-run-this-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before you run this notebook</a></span></li><li><span><a href="#Setting-up-training-/-validation-data" data-toc-modified-id="Setting-up-training-/-validation-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up training / validation data</a></span></li></ul></div>

# Before you run this notebook

Make sure that you have all of the appropriate packages installed. 
See the DeepBLAST README on installation instructions.
Also if you are using a GPU, make sure that CUDA and CUDNN are correctly configured.
In the slurm environment that was used to run this notebook, the following modules were loaded.

```
module load gcc cudnn cuda
```

This notebook is expected to take >15 minutes to run.

# Setting up training / validation data

Here we will show an example of how to run deepblast on a small subset of structural alignments generated from TM-align.

In [1]:
import os
import pandas as pd
from deepblast.sim import hmm_alignments
import argparse

fname = '../data/tm_align_output_10k.tab'
cols = [
    'chain1_name', 'chain2_name', 'tmscore1', 'tmscore2', 'rmsd',
    'chain1', 'chain2', 'alignment'
]
align_df = pd.read_table(fname, header=None, sep='\s+')
align_df.columns = cols
n_alignments = align_df.shape[0]

The simulated alignments will be split into training / testing and validation.

In [2]:
parts = n_alignments // 10
train_df = align_df.iloc[:parts * 8]
test_df = align_df.iloc[parts * 8:parts * 9]
valid_df = align_df.iloc[parts * 9:]

# save the files to disk.
if not os.path.exists('data'):
    os.mkdir('data')
    
train_df.to_csv('data/train.txt', sep='\t', index=None, header=None)
test_df.to_csv('data/test.txt', sep='\t', index=None, header=None)
valid_df.to_csv('data/valid.txt', sep='\t', index=None, header=None)

We will prepare the environment to make sure that the appropriate output directories exist to store the results.

In [3]:
from deepblast.trainer import DeepBLAST
from pytorch_lightning import Trainer

output_dir = 'struct_results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

We will now create the arguments.  First we need to download the Protrans model and tokenizer.

In [4]:
from transformers import T5EncoderModel, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

Some weights of the model checkpoint at Rostlab/prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.11.layer.2.DenseReluDense.wi.weight', 'decoder.block.17.layer.2.DenseReluDense.wi.weight', 'decoder.block.17.layer.0.SelfAttention.k.weight', 'decoder.block.2.layer.1.EncDecAttention.q.weight', 'decoder.block.12.layer.1.EncDecAttention.q.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.18.layer.0.SelfAttention.v.weight', 'decoder.block.20.layer.0.SelfAttention.q.weight', 'decoder.block.19.layer.0.SelfAttention.k.weight', 'decoder.block.7.layer.0.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.5.layer.1.EncDecAttention.o.weight', 'decoder.block.10.layer.2.layer_norm.weight', 'decoder.block.16.layer.1.EncDecAttention.q.weight', 'decoder.block.23.laye

Now let's create the DeepBLAST model.  

In [5]:
model = DeepBLAST(
    train_pairs=f'{os.getcwd()}/data/train.txt',  # training data
    test_pairs=f'{os.getcwd()}/data/test.txt',    # test data
    valid_pairs=f'{os.getcwd()}/data/valid.txt',  # validation data
    output_directory=output_dir,                  # output directory storing model + diagnostics
    batch_size=10,                                # number of alignments per training batch
    num_workers=30,                               # number of cores for manipulating training data
    layers=2,                                     # number of CNN layers for blosum matrix estimation
    learning_rate=5e-5,                           # learning rate
    loss='cross_entropy',                         # type of loss function 
    lm=model,                                     # pretrained language model 
    tokenizer=tokenizer                           # tokenizer for residues
)

We can now train the model.

In [6]:
trainer = Trainer(
    max_epochs=5,
    gpus=1,
    check_val_every_n_epoch=1,
    limit_train_batches=5,
    limit_val_batches=1
)

trainer.fit(model)

/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/lightning_lite/plugins/environments/slurm.py:167: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /mnt/ceph/users/jmorton/venv/deepblast/lib/python3. ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-pa

Sanity Checking: 0it [00:00, ?it/s]

/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPer

Training: 0it [00:00, ?it/s]

/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


...

/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Validation: 0it [00:00, ?it/s]

/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/mnt/ceph/users/jmorton/venv/deepblast/lib/python3.9/site-packages/numba/cuda/compiler.py:724: NumbaPerformanceWarning: Grid size (32) < 2 * SM count (216) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


The model diagnostics can be directly visualized in Tensorboard. Here, we show the losses, the accuracy and the alignment results.

In [7]:
!ls lightning_logs

version_0  version_1  version_2191072


In [8]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs